## Rod Cutting Problem

![prices](./data/rod-cutting.png)

In [1]:
from functools import lru_cache, wraps
from collections import defaultdict
import time

In [2]:
prices={i+1:v for i,v in enumerate([1,5,8,9,10,17,17,20,24,30])}

In [3]:
print(prices[2],prices[9])

5 24


In [4]:
def revenue(r):
    return max([prices[r]]+[(revenue(i)+revenue(r-i)) for i in range(1,r)])

In [5]:
revenue(9)

25

In [6]:
revenue(11)

KeyError: 11

In [7]:
prices=defaultdict(lambda : -float('inf'))
# 如果不在价格表里，则视为无穷小

In [8]:
prices[11]

-inf

In [9]:
for i,v in enumerate([1,5,8,9,10,17,17,20,24,30]):
    prices[i+1]=v

In [10]:
print(prices[2],prices[11])

5 -inf


In [11]:
revenue(10)

30

In [12]:
revenue(11)

31

In [13]:
start=time.time()
revenue(15)
end=time.time()
print('used time:{}'.format(end-start))

used time:11.927026510238647


In [17]:
# 慢，加缓存
cache={}
def revenue_cache(r):
    if r in cache:
        return cache[r]
    r_optimal=max([prices[r]]+[(revenue(i)+revenue(r-i)) for i in range(1,r)])
    cache[r]=r_optimal
    return r_optimal

In [20]:
start=time.time()
revenue_cache(15)
end=time.time()
print('used time:{}'.format(end-start))

used time:0.0008900165557861328


In [21]:
# python标准库lru_cache，不对原代码修改即可缓存
@lru_cache()
def revenue_lru(r):
    return max([prices[r]]+[(revenue(i)+revenue(r-i)) for i in range(1,r)])

In [22]:
start=time.time()
revenue_lru(15)
end=time.time()
print('used time:{}'.format(end-start))

used time:0.00047397613525390625


In [30]:
# lru_cache只能以字典形式存，如果需要存到list，需要自己实现memo
def memo(func):
    cache={}
    @wraps(func)
    def __wrap(*args,**kwargs):
        str_key=str(args)+str(kwargs)
        if str_key not in cache:
            result=func(*args,**kwargs)
            cache[str_key]=result
        return cache[str_key]
    return __wrap

In [41]:
solution={}

@memo
def revenue(r):
    split,r_star=max([(0,prices[r])]+[(i,revenue(i)+revenue(r-i)) for i in range(1,r)],key=lambda x:x[1])
    solution[r]=(split,r-split)
    return r_star

In [42]:
start=time.time()
revenue(15)
end=time.time()
print('used time:{}'.format(end-start))

used time:0.0009975433349609375


In [43]:
solution

{1: (0, 1),
 2: (0, 2),
 3: (0, 3),
 4: (2, 2),
 5: (2, 3),
 6: (0, 6),
 7: (1, 6),
 8: (2, 6),
 9: (3, 6),
 10: (0, 10),
 11: (1, 10),
 12: (2, 10),
 13: (3, 10),
 14: (2, 12),
 15: (2, 13)}

In [44]:
def parse_solution(r, revenue_solution):    
    left, right = revenue_solution[r]
    
    if left == 0: return [right]
    
    return [left] + parse_solution(right, revenue_solution)

In [45]:
def pretty_solution(splits):
    return ' -> '.join(map(str, splits))

pretty_solution(parse_solution(15, solution))

'2 -> 3 -> 10'

In [46]:
revenue(101)

301

## Edit Distance

![Levenshtein](./data/edit-distance.png)

In [47]:
@memo
def get_edit_distance(string1, string2):
    
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    return min(
        [get_edit_distance(string1[:-1], string2) + 1, 
         get_edit_distance(string1, string2[:-1]) + 1, 
         get_edit_distance(string1[:-1], string2[:-1]) + (0 if string1[-1] == string2[-1] else 2)]
    )
# 

In [49]:
get_edit_distance("Shanghai","Shanggai")

2